## Segmenting and Clustering Neighborhoods in Toronto using Foursqaure APIs
In this assignment, it is required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet.

For the Toronto neighborhood data, a Wikipedia page (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) exists that has all the information we need to explore and cluster the neighborhoods in Toronto. One will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, one can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

In [1]:
import requests # to create connection request
import lxml.html as lxml
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup  # import Beautifulsoup 

##  To get the postal code location from data wiki web page data


#### wiki location web page url

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
### to create connection request to wiki page 
page=requests.get(url).text

In [5]:
# BeautifulSoup obejct
soup=BeautifulSoup(page,'lxml')

## As we can see in wiki List_of_postal_codes_of_Canada page all location table comes under "table",{"class":"wikitable sortable"}

In [78]:
#print(soup.prettify()) to see the wiki page structure

In [6]:
location_table = soup.find("table",{"class":"wikitable sortable"}) 

#Create page objcet for wiki page
page = requests.get(url)

#Store the  website content under web_doc
web_doc = lxml.fromstring(page.content)

In [8]:
## check the web_doc content
web_doc

<Element html at 0x7f9af95706d8>

In [10]:
## collect the wiki location web data only for tr
tr_elements =web_doc.xpath('//tr')

In [11]:
#Check the length of the first 10 rows
[len(T) for T in tr_elements[:10]]


[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [12]:
## check the tr element structure
tr_elements[:5]

[<Element tr at 0x7f9af9570a48>,
 <Element tr at 0x7f9af9570e08>,
 <Element tr at 0x7f9af9519228>,
 <Element tr at 0x7f9af95194a8>,
 <Element tr at 0x7f9af9519408>]

In [15]:
#Create empty list to store wiki web location table data 
Dict=[]
colm=[]
i=0

#Store in first row, header and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    colm.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [16]:
colm

[('Postcode', []), ('Borough', []), ('Neighbourhood\n', [])]

In [18]:
#Header will  be columns name i.e ist row from the element and respective columns values are stored in  the second row onwards
for i in range(1,len(tr_elements)):
    # J is our i'th row
    J=tr_elements[i]
    
    #We can check row lenght if not 3 then the //tr data is not from location table
    if len(J)!=3:
        break
    
    #k will be the index of our column
    k=0
    
    #Iterating for each row 
    for j in J.iterchildren():
        data=j.text_content() 
        #Check if row is empty
        if k>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the k'th column
        colm[k][1].append(data)
        #Increment k for the next column
        k+=1    
[len(C) for (title,C) in colm]
Dict={title:column for (title,column) in colm}   # Dictionary with Postcodes, Borough and Neighbourhoods
df=pd.DataFrame(Dict)  # Creating the pandas Dataframe using the dictionary values

In [21]:
 #Dictionary with Postcodes, Borough and Neighbourhoods
Dict={title:column for (title,column) in colm} 

In [22]:
# Creating Dataframe for wiki location web page

df=pd.DataFrame(Dict)  

In [23]:
# view few columns from df dataframe.
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


## Now we have to perform next stage of EDA process called data cleansing and munging 

### we should check any null, data inconsistency ,fields forma values in the data frame and more information with user of varius command

In [25]:
## check the df data frame size and information about columns and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 3 columns):
Postcode          576 non-null object
Borough           576 non-null object
Neighbourhood
    576 non-null object
dtypes: object(3)
memory usage: 13.6+ KB


In [28]:
## Data frame columns
df.columns

Index(['Postcode', 'Borough', 'Neighbourhood\n'], dtype='object')

In [33]:
# few bottom data
df.tail()

,Postcode,Borough,Neighbourhood
571,M8Z,Etobicoke,Mimico NW\n
572,M8Z,Etobicoke,The Queensway West\n
573,M8Z,Etobicoke,Royal York South West\n
574,M8Z,Etobicoke,South of Bloor\n
575,M9Z,Not assigned,Not assigned\n


In [34]:
# few top data
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [35]:
df_Borough=df[df.Borough != 'Not assigned']

In [36]:
df_Borough.columns = ['Postcode', 'Borough', 'Neighbourhood']

In [47]:
df_Borough_new=df_Borough[['Postcode', 'Borough']].copy()

# Remove \n from the Neighbourhood column

df_Borough_new.loc[:,'Neighbourhood'] =pd.Series(df_Borough['Neighbourhood']).str.slice(stop=-1)

In [38]:
# check the newly data frame

df_Borough_new.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [39]:
df_Borough_new.loc[df_Borough_new.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df_Borough_new.Borough

In [40]:
 # if posatl code duplicate then add in the neighbourhoood with comma 
df_new=df_Borough_new.groupby('Postcode', sort=False)['Neighbourhood'].apply(','.join) 

In [41]:
# Check new data frame
df_new.head()

Postcode
M3A                                  Parkwoods,Parkwoods
M4A                    Victoria Village,Victoria Village
M5A    Harbourfront,Regent Park,Harbourfront,Regent Park
M6A    Lawrence Heights,Lawrence Manor,Lawrence Heigh...
M7A                            Queen's Park,Queen's Park
Name: Neighbourhood, dtype: object

In [42]:
## Create a new neighbourhood Data list
df_new_FINAL=pd.DataFrame({'Postcode':df_new.index, 'df_new':df_new.values})
df_new_F=df_new_FINAL.rename(columns={ df_new_FINAL.columns[1]: "Neighbourhood" })

df_N=df_new_F['Neighbourhood'].tolist()

In [43]:
# create a Borough lit
df_Borough_BONLY=df_Borough['Borough'] 
# create a postcode list
df_Borough_PONLY=df_Borough['Postcode'] 

In [44]:
df_Borough_BandP=pd.concat([df_Borough_PONLY, df_Borough_BONLY], axis=1)

# drop duplicates rows
df_Borough_BandP_new = df_Borough_BandP.drop_duplicates(subset=['Postcode','Borough'])  

In [45]:
# refresh the below lists
df_P=df_Borough_BandP_new['Postcode'].tolist() 
df_B=df_Borough_BandP_new['Borough'].tolist() 

In [46]:
# Create a dictionary with the use of above lists
Toronto_DICT= {'Postcode':df_P,'Borough':df_B, 'Neighbourhood':df_N} 

# create the dataframe 

Toronto_df=pd.DataFrame(Toronto_DICT) 

# Check the toronto data frame with top five rows
Toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,"Parkwoods,Parkwoods"
1,M4A,North York,"Victoria Village,Victoria Village"
2,M5A,Downtown Toronto,"Harbourfront,Regent Park,Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor,Lawrence Heigh..."
4,M7A,Queen's Park,"Queen's Park,Queen's Park"


## Phase 3 to get geocode for Toronto Nighbourhood with the use fo Geospatial data

In [48]:
# reading geospatial_data csv file

geo_file = 'http://cocl.us/Geospatial_data'
Col_headers = ["Postcode","Latitude","Longitude"]

 # getting latitude and longitude

df_Geo_Toronto = pd.read_csv(geo_file,names=Col_headers,skiprows=1) 

In [49]:
# view top rows from data frame

df_Geo_Toronto.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [51]:
# merging the dataframes to get the L & L data annexed to the earlier DataFrame

df_final=pd.merge(Toronto_df, df_Geo_Toronto, on='Postcode', how='inner')  

In [52]:
# view top rows from final data frame
df_final.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,"Parkwoods,Parkwoods",43.753259,-79.329656
1,M4A,North York,"Victoria Village,Victoria Village",43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park,Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor,Lawrence Heigh...",43.718518,-79.464763
4,M7A,Queen's Park,"Queen's Park,Queen's Park",43.662301,-79.389494


## Pahse 4 Data Visualization from FourSqaure API

In [53]:
# This is the 3rd section (Clustering & Neighbourthood analysis.....)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

In [54]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [57]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

df_Toronto_Special=df_final[df_final['Borough'].str.contains("Toronto")]   # select Boroughs with "Toronto"

In [58]:
# Shape of the Toronto Data Frame
df_Toronto_Special.shape

(38, 5)

In [59]:
# view the fews rows from data frame
df_Toronto_Special.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park,Harbourfront,Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson,Garden District,Ryerson,Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,"St. James Town,St. James Town",43.651494,-79.375418
19,M4E,East Toronto,"The Beaches,The Beaches",43.676357,-79.293031
20,M5E,Downtown Toronto,"Berczy Park,Berczy Park",43.644771,-79.373306


## Get the Latitude and Longitude for the Toronto Ontario

In [60]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [61]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto_Special['Latitude'], df_Toronto_Special['Longitude'], df_Toronto_Special['Borough'], df_Toronto_Special['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [62]:
neighborhoods=df_Toronto_Special

#CT_data = neighborhoods[neighborhoods['Borough'] == 'Central Toronto'].reset_index(drop=True)

CT_data = neighborhoods.reset_index(drop=True)
CT_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park,Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District,Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,"St. James Town,St. James Town",43.651494,-79.375418
3,M4E,East Toronto,"The Beaches,The Beaches",43.676357,-79.293031
4,M5E,Downtown Toronto,"Berczy Park,Berczy Park",43.644771,-79.373306


In [63]:
address = 'Central Toronto, Ontario'  # selecting central Toronto (CT) for deep dive

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_CT = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(CT_data['Latitude'], CT_data['Longitude'], CT_data['Borough'], CT_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_CT)  
    
map_CT

The geograpical coordinate of Central Toronto are 43.6449033, -79.3818364.


In [70]:
from IPython.display import HTML
from IPython.display import display

# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click <a href="javascript:code_toggle()">here</a>.''')
display(tag)

CLIENT_ID = 'KYCVEGQAE10NBJCDUYK0ESKSNKZVT2ATJJMA5CXE25EXIJ5F' # your Foursquare ID
CLIENT_SECRET = 'LXEF0JTXJ1MZVOHWH5GIUGRSVVMEAHXPRKNTOIKLFZ0N1ZHY' # your Foursquare Secret
VERSION = '20180604'

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [65]:
# check the Shape of  ct_data Data Frame
CT_data.shape

(38, 5)

In [66]:
CT_data.loc[0, 'Neighbourhood']

'Harbourfront,Regent Park,Harbourfront,Regent Park'

In [71]:
neighborhood_latitude = CT_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = CT_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = CT_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

results = requests.get(url).json()
results

Latitude and longitude values of Harbourfront,Regent Park,Harbourfront,Regent Park are 43.6542599, -79.3606359.


{'meta': {'code': 200, 'requestId': '5cc78d9b787dba002b4c306f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 48,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [72]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
5,Impact Kitchen,Restaurant,43.656369,-79.356980
6,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
7,Dominion Pub and Kitchen,Pub,43.656919,-79.358967
8,Corktown Common,Park,43.655618,-79.356211
9,The Distillery Historic District,Historic Site,43.650244,-79.359323


## Make Cluster for Toronto Niegnhbourhood
#### 5.1 Explore Nighbourhood in Toronto

In [74]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        url
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
#        results = requests.get(url).json()["response"]['venues']
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
#            v['name'], 
#            v['location']['lat'], 
#            v['location']['lng'],  
#            v['categories'][0]['name']) for v in results])
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [75]:
CT_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park,Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District,Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,"St. James Town,St. James Town",43.651494,-79.375418
3,M4E,East Toronto,"The Beaches,The Beaches",43.676357,-79.293031
4,M5E,Downtown Toronto,"Berczy Park,Berczy Park",43.644771,-79.373306


In [76]:
df_Toronto_Special.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park,Harbourfront,Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson,Garden District,Ryerson,Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,"St. James Town,St. James Town",43.651494,-79.375418
19,M4E,East Toronto,"The Beaches,The Beaches",43.676357,-79.293031
20,M5E,Downtown Toronto,"Berczy Park,Berczy Park",43.644771,-79.373306


In [77]:
CT_venues = getNearbyVenues(names=CT_data['Borough'],
                                   latitudes=CT_data['Latitude'],
                                   longitudes=CT_data['Longitude']
                                  )

Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Downtown Toronto
West Toronto
Central Toronto
Downtown Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto


In [78]:
CT_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Downtown Toronto,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Downtown Toronto,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Downtown Toronto,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Downtown Toronto,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [79]:

# one hot encoding
CT_onehot = pd.get_dummies(CT_venues[['Venue Category']], prefix="", prefix_sep="")

In [80]:
## shape of Data frame
CT_onehot.shape

(1706, 235)

In [81]:
# add Borough column back to dataframe
#CT_onehot.loc[:,'Neighborhood'] = pd.Series(CT_venues['Neighborhood']) 
CT_onehot['Neighborhood'] = CT_venues['Neighborhood']

In [82]:
CT_onehot.columns.get_loc("Neighborhood")

160

In [83]:
# move neighborhood column to the first column
CT_onehot.columns[CT_onehot.columns.get_loc("Neighborhood")]

'Neighborhood'

In [84]:
list(CT_onehot.columns[:-CT_onehot.columns.get_loc("Neighborhood")])

['Adult Boutique',
 'Afghan Restaurant',
 'Airport',
 'Airport Food Court',
 'Airport Gate',
 'Airport Lounge',
 'Airport Service',
 'Airport Terminal',
 'American Restaurant',
 'Antique Shop',
 'Aquarium',
 'Argentinian Restaurant',
 'Art Gallery',
 'Art Museum',
 'Arts & Crafts Store',
 'Asian Restaurant',
 'Athletics & Sports',
 'Auto Workshop',
 'BBQ Joint',
 'Baby Store',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Baseball Stadium',
 'Basketball Stadium',
 'Beach',
 'Beer Bar',
 'Beer Store',
 'Belgian Restaurant',
 'Bistro',
 'Board Shop',
 'Boat or Ferry',
 'Bookstore',
 'Boutique',
 'Brazilian Restaurant',
 'Breakfast Spot',
 'Brewery',
 'Bubble Tea Shop',
 'Building',
 'Burger Joint',
 'Burrito Place',
 'Bus Line',
 'Butcher',
 'Café',
 'Cajun / Creole Restaurant',
 'Camera Store',
 'Caribbean Restaurant',
 'Cheese Shop',
 'Chinese Restaurant',
 'Chocolate Shop',
 'Church',
 'Climbing Gym',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'College Arts Building',
 'Colle

In [85]:
fixed_columns = [CT_onehot.columns[-1]] + list(CT_onehot.columns[:-1])
fixed_columns

['Yoga Studio',
 'Adult Boutique',
 'Afghan Restaurant',
 'Airport',
 'Airport Food Court',
 'Airport Gate',
 'Airport Lounge',
 'Airport Service',
 'Airport Terminal',
 'American Restaurant',
 'Antique Shop',
 'Aquarium',
 'Argentinian Restaurant',
 'Art Gallery',
 'Art Museum',
 'Arts & Crafts Store',
 'Asian Restaurant',
 'Athletics & Sports',
 'Auto Workshop',
 'BBQ Joint',
 'Baby Store',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Baseball Stadium',
 'Basketball Stadium',
 'Beach',
 'Beer Bar',
 'Beer Store',
 'Belgian Restaurant',
 'Bistro',
 'Board Shop',
 'Boat or Ferry',
 'Bookstore',
 'Boutique',
 'Brazilian Restaurant',
 'Breakfast Spot',
 'Brewery',
 'Bubble Tea Shop',
 'Building',
 'Burger Joint',
 'Burrito Place',
 'Bus Line',
 'Butcher',
 'Café',
 'Cajun / Creole Restaurant',
 'Camera Store',
 'Caribbean Restaurant',
 'Cheese Shop',
 'Chinese Restaurant',
 'Chocolate Shop',
 'Church',
 'Climbing Gym',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'College Arts Bu

In [86]:
CT_onehot = CT_onehot[fixed_columns]

CT_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Downtown Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0

In [87]:
CT_onehot.shape

(1706, 235)

In [88]:
CT_grouped = CT_onehot.groupby('Neighborhood').mean().reset_index()
CT_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Central Toronto,0.008850,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017699,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008850,0.000000,0.008850,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008850,0.008850,0.000000,0.000000,0.026549,0.000000,0.00885,0.000000,0.044248,0.000000,0.000000,0.000000,0.000000,0.008850,0.000000,0.000000,0.000000,0.026549,0.000000,0.079646,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008850,0.008850,0.000000,0.00

In [89]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [90]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = CT_grouped['Neighborhood']

for ind in np.arange(CT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(CT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Dessert Shop,Sushi Restaurant,Gym,Pub,Burger Joint
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Seafood Restaurant
2,East Toronto,Coffee Shop,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Café,Park,Brewery,American Restaurant,Bakery,Pizza Place
3,West Toronto,Bar,Café,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Bakery,Diner,Breakfast Spot,Pharmacy


In [91]:
# set number of clusters
kclusters = 4

CT_grouped_clustering = CT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 2, 1], dtype=int32)

In [92]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [93]:
CT_merged = CT_data
CT_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park,Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District,Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,"St. James Town,St. James Town",43.651494,-79.375418
3,M4E,East Toronto,"The Beaches,The Beaches",43.676357,-79.293031
4,M5E,Downtown Toronto,"Berczy Park,Berczy Park",43.644771,-79.373306


In [94]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Central Toronto,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Dessert Shop,Sushi Restaurant,Gym,Pub,Burger Joint
1,3,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,American Restaurant,Seafood Restaurant
2,2,East Toronto,Coffee Shop,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Café,Park,Brewery,American Restaurant,Bakery,Pizza Place
3,1,West Toronto,Bar,Café,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Bakery,Diner,Breakfast Spot,Pharmacy


In [95]:
CT_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park,Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District,Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,"St. James Town,St. James Town",43.651494,-79.375418
3,M4E,East Toronto,"The Beaches,The Beaches",43.676357,-79.293031
4,M5E,Downtown Toronto,"Berczy Park,Berczy Park",43.644771,-79.373306


In [96]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
CT_merged = CT_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Borough')

## Data Visualization for respective cluster


In [97]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(CT_merged['Latitude'], CT_merged['Longitude'], CT_merged['Borough'], CT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters